# Model Exploration Dash Prototype 

In [6]:
## Imports
#

import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import callback_context
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output, State
import pandas as pd
import os, ntpath
import numpy as np
from functools import reduce
import numpy as np
import gillespy2
from gillespy2 import Model, Species, Reaction, Parameter, RateRule, AssignmentRule, FunctionDefinition
from gillespy2 import EventAssignment, EventTrigger, Event
from sciope.stochmet.stochmet import StochMET


/Users/mgeiger/.pyenv/versions/3.8.2/envs/dash-playtest/lib/python3.8/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.manifold.t_sne module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.manifold. Anything that cannot be imported from sklearn.manifold is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
## GillesPy2 Model
#

class Dimerization(Model):
    def __init__(self, parameter_values=None):
        Model.__init__(self, name="Dimerization")
        self.volume = 1

        # Parameters
        self.add_parameter(Parameter(name="k_c", expression=0.005))
        self.add_parameter(Parameter(name="k_d", expression=0.08))

        # Species
        self.add_species(Species(name="Monomer", initial_value=30, mode="discrete"))
        self.add_species(Species(name="Dimer", initial_value=0, mode="discrete"))

        # Reactions
        self.add_reaction(Reaction(name="r_creation", reactants={'Monomer': 2}, products={'Dimer': 1}, rate=self.listOfParameters["k_c"]))
        self.add_reaction(Reaction(name="r_dissociation", reactants={'Dimer': 1}, products={'Monomer': 2}, rate=self.listOfParameters["k_d"]))

        # Timespan
        self.timespan(np.linspace(0, 100, 101))
        
model = Dimerization()

In [3]:
from sciope.utilities.gillespy2 import wrapper
settings = {"number_of_trajectories":1}
simulator = wrapper.get_simulator(gillespy_model=model, run_settings=settings, species_of_interest=['Monomer', 'Dimer'])
expression_array = wrapper.get_parameter_expression_array(model)

In [4]:
from dask.distributed import Client
#from sciope.designs import latin_hypercube_sampling
#from sciope.utilities.summarystats.auto_tsfresh import SummariesTSFRESH
import sciope.utilities as util
import sciope.designs as designs
from sciope.utilities.priors.uniform_prior import UniformPrior

sampler = UniformPrior(expression_array*0.8, expression_array*1.5)

In [8]:
c = Client()

/Users/mgeiger/.pyenv/versions/3.8.2/envs/dash-playtest/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 54762 instead
  warnings.warn(


In [14]:
from sciope.utilities.summarystats.summary_base import SummaryBase

summaries = SummaryBase('summaries')
summaries.compute = lambda x: x

met = StochMET(simulator, sampler, summaries)

In [15]:
met.compute(n_points=20)

In [44]:
#First lets add some appropiate information about the model and features
met.data.configurations['parameter_labels'] = list(model.listOfParameters.keys())
met.data.configurations['species_labels'] = list(model.listOfSpecies.keys())
met.data.configurations['timepoints'] = model.tspan

In [81]:
data = met.data
data

In [123]:
import json

In [ ]:
#### Start of Dash Setup
##

In [82]:
## Defaults
#

default_num_groups = 5
default_trajectory_selector = 'slider' # either 'dropdown' or 'slider'

In [83]:
## Helpers
#

def options_factory(iterable, label_accessor=lambda x: x, value_accessor=lambda x: x):
    """
    Factory function for creating a dash dropdown menu's 'options' attribute.

    Positional arguments:
    iterable -- any iterable data structure.

    Keyword arguments (optional):
    label_accessor -- function to retrieve the label from a single data point in iterable.
    value_accessor -- function to retrieve the value from a single data point in iterable.

    Returns a list of dictionaries with keys 'label' and 'value'.
    """
    return [
        { 'label': label_accessor(d), 'value': value_accessor(d) }
        for d in iterable
    ]

def get_zeroed_df(df):
    zero_df = df - df
    zero_df['time'] = df['time'].copy()
    return zero_df


def default_local_store(dfs, num_groups):
    """
    Initial client-side JSON storage for callbacks to refernece.
    """
    return {
        'dfs': [ df.to_json(orient='split') for df in dfs ],
        'group_assigns' : [None] * len(dfs),
        'group_data' : [get_zeroed_df(dfs[0]).to_json(orient='split')] * default_num_groups
    }

In [84]:
## Read the data
#

#results_dir = './results_csv_06042020_150109/'
#dfs = read_ensemble_data_from_results_folder(results_dir)

In [87]:
## Setup menus and styling for dropdowns
#
'''
# Extra styling for dropdowns
dropdown_styles = {
}

# Setup species dropdown options
all_species = [ c for c in dfs[0].columns if c != 'time' ]
species_options = options_factory(all_species)
species_options.insert(0, options_factory(['All'])[0])

# Group dropdown options
group_options = options_factory(
    # 0-indexed internally, labeled as 1-indexed
    range(default_num_groups), label_accessor=lambda x: str(x+1)
)

# Trajectory dropdown options
trajectory_options = options_factory(
    # 0-indexed internally, labeled as 1-indexed
    range(len(dfs)), label_accessor=lambda x: str(x+1)
)
'''

"\n# Extra styling for dropdowns\ndropdown_styles = {\n}\n\n# Setup species dropdown options\nall_species = [ c for c in dfs[0].columns if c != 'time' ]\nspecies_options = options_factory(all_species)\nspecies_options.insert(0, options_factory(['All'])[0])\n\n# Group dropdown options\ngroup_options = options_factory(\n    # 0-indexed internally, labeled as 1-indexed\n    range(default_num_groups), label_accessor=lambda x: str(x+1)\n)\n\n# Trajectory dropdown options\ntrajectory_options = options_factory(\n    # 0-indexed internally, labeled as 1-indexed\n    range(len(dfs)), label_accessor=lambda x: str(x+1)\n)\n"

In [151]:
## App
#

external_stylesheets = [
    {
        'href': 'https://stackpath.bootstrapcdn.com/bootstrap/4.1.3/css/bootstrap.min.css',
        'rel': 'stylesheet',
        'integrity': 'sha384-MCw98/SFnGE8fJT3GXwEOngsV7Zt27NXFoaoApmYm81iuXoPkFOJwJ8ERdknLPMO',
        'crossorigin': 'anonymous'
    }
]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

In [152]:
## Fragments
#

# Initial client JSON store
#store = default_local_store(dfs, default_num_groups)

In [153]:
## Components
#

In [154]:
data

In [155]:
## Model Exploration Layout

fig = px.scatter(x=data.x[:,0], y=data.x[:,1])

model_exploration_layout = html.Div([
    html.Div([ dcc.Graph(id='parameter-space-graph', figure=fig) ],
             id='parameter-space-graph-wrapper',
             className='col-md-12',
    ),
    html.Div([ dcc.Graph(id='output') ])
], className='card card-body')


In [182]:
def default_local_store(data):
    """
    Initial client-side JSON storage for callbacks to refernece.
    """
    return {
        'params': data.x.tolist(),
        'timepoints': data.configurations['timepoints'].tolist(),
        'timeseries' : data.ts.tolist(),
    }

In [183]:
data.ts.tolist()[:,0]

TypeError: list indices must be integers or slices, not tuple

In [168]:
'''
{
  "points": [
    {
      "curveNumber": 0,
      "pointNumber": 6,
      "pointIndex": 6,
      "x": 0.0048327475341438005,
      "y": 0.0703783913529779
    }
  ]
}
'''
@app.callback(
    Output('output', 'figure'),
    [Input('parameter-space-graph', 'clickData')],
    [State('memory', 'data')])
def display_time_series(clickData, data):
    try:
        p = clickData['points'][0]
        x, y = p['x'], p['y']
        params = data['params']
        index = [ d for d in params if d[0] == x ][0]
        ts_all = data['timeseries'][index]
        ts_0 = ts_all[:,0]
        return px.scatter(x=data.configurations['timepoints'], y=ts_0)
    except:
        return px.scatter(x=data.configurations['timepoints'])

In [169]:
## App Layout
#

app.layout = html.Div([ model_exploration_layout
    #dcc.Store(id='memory', data=store),
    #group_assigner_layout,
    #group_inspector_layout
],
style={
    'font-family': 'Arial, Helvetica, sans-serif'
})

In [170]:
## Run
#

app.run_server(mode='inline', debug=True)